In [1]:
import json
import websockets, asyncio

# the interface to the server
class WSManager:
    def __init__(self):
        self.debug = False
        self.inQueue = asyncio.Queue()
        self.outQueue = asyncio.Queue()
        self.ws = None
        asyncio.create_task(self.message_sender_loop())

    async def message_sender_loop(self):
        while True:
            try:
                message = await self.outQueue.get()
                await self.ws.send(json.dumps(message, indent=4))
            except Exception as e: print(e)
    def recv(self,message):
        self.inQueue.put_nowait(message)
        if self.debug:
            print("recv: ",message)
    def send(self,command, content):
        self.outQueue.put_nowait({'command':command,'content':content})
    
ws = WSManager()
ws.debug = True

# the server
def server():
    async def echo(websocket):
        ws.ws = websocket
        print('connect')
        async for message in websocket:
            try:
                ws.recv(json.loads(message))
            except json.decoder.JSONDecodeError:
                ws.recv(message)

    async def main():
        try:
            async with websockets.serve(echo, "localhost", 8765):
                await asyncio.Future()  # run forever
        except websockets.exceptions.ConnectionClosedError as e: print(e)

    asyncio.run(main())

# start the server in a separate thread to avoid blocking
import threading
t=threading.Thread(target=server)
t.start()

connect
recv:  {'command': 'state', 'content': {'baseLinkPos': {'x': -0.11361023, 'y': -0.0467397869}, 'baseLinkOrientation': 0.106178574, 'baseLinkVelocity': {'x': -0.09783887, 'y': 0.000716104754}, 'baseLinkAngularVelocity': 0.0006899055, 'wheelBaseOrientation': [3.52815151, 3.78001571, 3.96643329, 3.97851849], 'wheelSpeed': [-0.0274148826, -0.0295335054, -0.03515495, -0.0213933643]}}


In [7]:
ws.send("action",{"voltage":[1,0,0,0,100,200,100,100]})

recv:  {'command': 'state', 'content': {'baseLinkPos': {'x': -1.38703692, 'y': -0.04704912}, 'baseLinkOrientation': 6.2034955, 'baseLinkVelocity': {'x': -0.122101218, 'y': -0.00127125042}, 'baseLinkAngularVelocity': -0.001359699, 'wheelBaseOrientation': [3.21128273, 4.08005428, 3.55027866, 3.46296883], 'wheelSpeed': [-0.0283784755, -0.0221891347, -0.03365396, -0.04376701]}}


In [8]:
ws.inQueue

<Queue at 0x1cf40042130 maxsize=0 _queue=[{'command': 'state', 'content': {'baseLinkPos': {'x': -0.11361023, 'y': -0.0467397869}, 'baseLinkOrientation': 0.106178574, 'baseLinkVelocity': {'x': -0.09783887, 'y': 0.000716104754}, 'baseLinkAngularVelocity': 0.0006899055, 'wheelBaseOrientation': [3.52815151, 3.78001571, 3.96643329, 3.97851849], 'wheelSpeed': [-0.0274148826, -0.0295335054, -0.03515495, -0.0213933643]}}, {'command': 'state', 'content': {'baseLinkPos': {'x': -0.462669343, 'y': -0.0469363146}, 'baseLinkOrientation': 0.153078645, 'baseLinkVelocity': {'x': -0.118741542, 'y': 0.00223530922}, 'baseLinkAngularVelocity': 0.000225561627, 'wheelBaseOrientation': [3.458417, 3.82383633, 3.9247613, 3.923528], 'wheelSpeed': [-0.0334638245, -0.025807431, -0.0218893886, -0.0213501528]}}, {'command': 'state', 'content': {'baseLinkPos': {'x': -0.7857292, 'y': -0.04697624}, 'baseLinkOrientation': 0.135071978, 'baseLinkVelocity': {'x': -0.118114874, 'y': -0.000454245135}, 'baseLinkAngularVelocit

In [6]:
import numpy as np
import torch
def flatten(list_of_lists):
    if len(list_of_lists) == 0:
        return list(list_of_lists)
    if hasattr(list_of_lists[0], '__iter__'):
        return flatten(list_of_lists[0]) + flatten(list_of_lists[1:])
    return list(list_of_lists[:1]) + flatten(list_of_lists[1:])
def decomposeCosSin(angle):
    return [np.cos(angle), np.sin(angle)]
def processFeature(state:dict,targetPos):
    feature = []
    feature.append(state['baseLinkPos']['x']-targetPos['x'])
    feature.append(state['baseLinkPos']['y']-targetPos['y'])
    feature.append(decomposeCosSin(state['baseLinkOrientation']))
    feature.append(state['baseLinkVelocity']['x'])
    feature.append(state['baseLinkVelocity']['y'])
    feature.append(state['baseLinkAngularVelocity'])
    feature.append(decomposeCosSin(state['wheelBaseOrientation']))
    feature.append(state['wheelSpeed'])
    feature = flatten(feature)
    return feature

In [40]:
feature = processFeature(ws.inQueue.get_nowait()['content'],{'x':0,'y':0})

In [10]:
from torch import nn
class Q(nn.Module):
    def __init__(self,state_size,action_size,hidden_size):
        self.state_size = state_size
        self.action_size = action_size
        self.fc = nn.Sequential(
        nn.Linear(state_size+action_size,hidden_size),
        nn.LeakyReLU(),
        nn.Linear(hidden_size,hidden_size),
        nn.LeakyReLU(),
        nn.Linear(hidden_size,1)
        )

    def forward(self,state,action):
        return self.fc(torch.cat([state,action],dim=1))

class Policy(nn.Module):
    def __init__(self,state_size,action_size,hidden_size):
        self.state_size = state_size
        self.action_size = action_size
        self.fc = nn.Sequential(
        nn.Linear(state_size,hidden_size),
        nn.LeakyReLU(),
        nn.Linear(hidden_size,hidden_size),
        nn.LeakyReLU(),
        nn.Linear(hidden_size,action_size)
        )

    def forward(self,state):
        return self.fc(state)